In [1]:
import pandas as pd
import numpy as np 
import scipy as sp
import matplotlib.pylab as plt

## A simple multivariate-normal-distribution implementation for anomaly detection of the driving patterns



In [5]:
df = pd.read_csv('dataset.csv')
df.columns

Index([u'id', u'timestamp', u'session_id', u'user_id', u'pc_throttle',
       u'pc_brake', u'pc_steering', u'pc_rpm', u'pc_speed', u'pc_pos_x',
       u'pc_pos_y', u'pc_pos_z', u'pc_laptime', u'pc_race_state',
       u'pc_lap_number', u'pc_lap_distance', u'vr_pos_x', u'vr_pos_y',
       u'vr_pos_z', u'vr_rotation_x', u'vr_rotation_y', u'vr_rotation_z',
       u'logitech_acceleration', u'logitech_brake', u'logitech_steering'],
      dtype='object')

In [6]:
float_cols = [i for i in df.dtypes.index if df.dtypes[i] == 'float64']
cat_cols = [i for i in df.dtypes.index if df.dtypes[i] == 'int64']
other_cols = list(set(df.columns) - (set(float_cols) | set(cat_cols)))

In [174]:
### multivariate normal distribution on the floats
df_flt = df[float_cols].copy()
df_flt.drop(['pc_lap_distance'],inplace=True,axis = 1)
float_cols = df_flt.columns

df_flt.reset_index(inplace = True,drop=True)

df_cat = df[cat_cols].copy() 
drivers = df_cat['user_id'].unique()
session = df_cat['session_id'].unique()
mean, std = df_flt.mean(), df_flt.std()
cov = df_flt.cov()

#feature space dim 
num_features,num_samples = len(float_cols), len(df_flt)

In [175]:
df_flt.columns

Index([u'pc_throttle', u'pc_brake', u'pc_steering', u'pc_rpm', u'pc_speed',
       u'pc_pos_x', u'pc_pos_y', u'pc_pos_z', u'pc_laptime', u'vr_pos_x',
       u'vr_pos_y', u'vr_pos_z', u'vr_rotation_x', u'vr_rotation_y',
       u'vr_rotation_z'],
      dtype='object')

In [176]:
def unnormalized_covariance(A,mu):
    """
    
    :param A: the design matrix A in R^{MxN} where each row is a separate sample point and the columns represent
    the dimension
    :param mu: a R^Nx1 vector with means 
    :return: 
    """
    # A is the design matrix     
    (M,N) = A.shape
    mean = np.matlib.repmat(mu,M,1)
    A = A-mean # desing matrix with subtracted out mean! 
    return np.matmul(A.T,A), N

def MVN_prob(x,mu, Pr,dPr):
    """
    calculate the p-value of X for a multivariate normal distribution 
    :param x: the feature vector  
    :param mu: the means 
    :param Pr: the precision matrix  (the iverse of the covariance matix!)
    :param dPr: the determinant of the precision matrix 
    :return: 
    """
    assert Pr.shape[0] == Pr.shape[1]
    assert x.shape == mu.shape 
    assert max(x.shape) == Pr.shape[0]
    
    N = max(x.shape) # dimensions of the feature vector! 
    n = min(x.shape)
    
    x.shape = (N,n)
    mu.shape = (N,n)
    
    # check for bad data ! 
    assert n == 1 , " invalid data format - x must be a rank 1 tensor " 
    
    x = x.reshape([N,1])
    N = len(x)
    return np.sqrt(dPr/(2*np.pi)**N)\
           *np.exp(-0.5*np.matmul(np.matmul((x-mu).T,Pr),x))

In [177]:
x = np.array([[0, 2], [1, 1], [2, 0]])
dff = pd.DataFrame(x)
cov,N = unnormalized_covariance(dff.as_matrix(),dff.mean().as_matrix())

In [178]:
# normalize the data // different scales lead to cancellation 
data = df_flt.copy()
mu = data.mean().as_matrix()
original_means = np.matlib.repmat(mu,data.shape[0],1)
data = data-original_means

In [180]:
#### 
data['pval'] = 0. 

# M is the number of samples per update! 
M = num_features*10
p_val = 5./100. # 5 percent

num_epochs = int(num_samples/M)
singular_mtx_counter = 0 
for epoch in range(0,num_epochs):
    interval = range(epoch*M,M*(epoch+1))
    # design matirx
    A = data.ix[interval,float_cols].as_matrix()
    mu = data.ix[interval,float_cols].mean().as_matrix()
    cov,N = unnormalized_covariance(A,mu)
    cov /= N # for the maximum likelihood? estimator! 
    mu.shape = (N,1)
    
    
    Pr = np.linalg.inv(cov)
    dPr = np.linalg.det(Pr)
    
    if dPr == 0:
        print "singular matrix?!"
        singular_mtx_counter +=1 
        continue
        
    ### make fast !!!! 
    for t in interval:
        x =  data.ix[t,float_cols].as_matrix()
        x.shape = (N,1)
        pval = MVN_prob(x,mu, Pr,dPr)
        data.ix[t,'pval'] = pval

/home/kenny/programs/anaconda2/envs/scicomp/lib/python2.7/site-packages/ipykernel/__main__.py:39: RuntimeWarning: invalid value encountered in sqrt
/home/kenny/programs/anaconda2/envs/scicomp/lib/python2.7/site-packages/ipykernel/__main__.py:39: RuntimeWarning: overflow encountered in exp


LinAlgError: Singular matrix

In [ ]:
data.ix[t,float_cols].as_matrix()

array([  5.81460279e-01,  -1.37434021e-01,  -1.31094476e-02,
         2.77393271e+03,   4.42206354e+01,   1.64556071e+02,
         9.18128468e+00,   3.34073239e+01,   4.31910477e+01,
                    nan,  -1.95950477e+00,  -1.11360512e+00,
         2.60612827e-02,  -4.36728296e-01,   4.55864772e-01,
         6.00823756e-01])

In [159]:
df_flt

array([[ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ],
       [ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ],
       [ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ],
       ..., 
       [ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ],
       [ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ],
       [ 0.41853972,  0.13743402,  0.01194615, ..., -0.52324877,
        -0.01152576,  0.        ]])